In [1]:
import pyautogui
import cv2
import mediapipe as mp
import numpy as np
import time

print("Libraries imported successfully")


Libraries imported successfully


In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [3]:
s = pyautogui.size()

In [4]:
# initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.flip(frame, 1)

    cv2.imshow('Output', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [5]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.flip(frame, 1)

    x,y,c = frame.shape
    result = hands.process(frame)

    if result.multi_hand_landmarks:
        handLandmarks = result.multi_hand_landmarks[0]

        mpDraw.draw_landmarks(frame, handLandmarks, mpHands.HAND_CONNECTIONS)


    cv2.imshow('Output', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
    

-1

In [6]:
cap  = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.flip(frame, 1)

    x,y,c = frame.shape
    result = hands.process(frame)

    if result.multi_hand_landmarks:
        handLandmarks = result.multi_hand_landmarks[0]

        index_x = int(handLandmarks.landmark[8].x * s[0])
        index_y = int(handLandmarks.landmark[8].y * s[1])

        pyautogui.moveTo(index_x, index_y, _pause=False)

        mpDraw.draw_landmarks(frame, handLandmarks, mpHands.HAND_CONNECTIONS)


    cv2.imshow('Output', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



-1

In [8]:
def normalized_distance(handslms, landmark1, landmark2):
    x1, y1 = handslms.landmark[landmark1].x, handslms.landmark[landmark2].y
    x2, y2 = handslms.landmark[landmark2].x, handslms.landmark[landmark2].y

    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def is_pinching(handslms, threshold=0.05):
    distance = normalized_distance(handslms, 4, 8)
    return distance < threshold

def advanced_pinch_detection(handslms, threshold=0.04):
    """
    More sophisticated pinch detection considering finger angles
    """
    # Check if thumb and index are extended
    thumb_tip = handslms.landmark[4]
    thumb_mcp = handslms.landmark[2]  # Thumb MCP joint
    index_tip = handslms.landmark[8]
    index_pip = handslms.landmark[6]  # Index PIP joint
    
    # Check if fingers are extended (simple check)
    thumb_extended = thumb_tip.y < thumb_mcp.y
    index_extended = index_tip.y < index_pip.y
    
    # Only consider pinch if both fingers are somewhat extended
    if thumb_extended and index_extended:
        distance = normalized_distance(handslms, 4, 8)
        return distance < threshold
    
    return False


In [9]:
cap  = cv2.VideoCapture(0)
last_click = 0
while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.flip(frame, 1)

    x,y,c = frame.shape
    result = hands.process(frame)

    if result.multi_hand_landmarks:
        handLandmarks = result.multi_hand_landmarks[0]

        index_x = int(handLandmarks.landmark[8].x * s[0])
        index_y = int(handLandmarks.landmark[8].y * s[1])
        thumb_x = int(handLandmarks.landmark[4].x * s[0])
        thumb_y = int(handLandmarks.landmark[4].y * s[1])

        pyautogui.moveTo(index_x, index_y, _pause=False)

        cv2.line(frame, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 0), 3)

        if is_pinching(handLandmarks) and time.time() - last_click > 0.5:
            pyautogui.click()
            last_click = time.time()

        
        mpDraw.draw_landmarks(frame, handLandmarks, mpHands.HAND_CONNECTIONS)


    cv2.imshow('Output', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

In [10]:
def sum_dict(dict):
    result = 0
    for key in dict:
        result+= dict[key]
    return result

In [11]:
cap  = cv2.VideoCapture(0)
last_click = 0
while True:

    # capture webcame frame and shape (width and height)
    ret, frame = cap.read()

    if not ret:
        break

    # flip frame for mirror effect
    frame = cv2.flip(frame, 1)

    h, w, c = frame.shape

    # convert from bgr to rgb for mediapipe, this is optional
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # have mediapipe hands predict hand landmarks
    result = hands.process(rgb_frame)

    # dict of fingers where key is landmark and value is 1 or 0 (detected or not)
    fingers = {"4": 0, "8": 0, "12": 0, "16": 0, "20": 0}


    # check if predicted landmarks were detected
    # if so draw & output them to the opencv window
    if result.multi_hand_landmarks:
        
        handLandmarks = result.multi_hand_landmarks[0]
        # thumb coordinates

        # a note here is we have to scale coordinates relative to the frame (height, width)
        x4, y4 = int(handLandmarks.landmark[4].x * w), int(handLandmarks.landmark[4].y * h)
        x2, y2 = int(handLandmarks.landmark[2].x * w), int(handLandmarks.landmark[2].y * h)

        # check if thumb is up
        if x4 > x2:
            fingers['4'] = 1
        
        # index finger coordinates
        x8, y8 = int(handLandmarks.landmark[8].x * w), int(handLandmarks.landmark[8].y * h)
        x6, y6 = int(handLandmarks.landmark[6].x * w), int(handLandmarks.landmark[6].y * h)

        if y8 < y6:
            fingers['8'] = 1
        
        # middle finger
        x12, y12 = int(handLandmarks.landmark[12].x * w), int(handLandmarks.landmark[12].y * h)
        x10, y10 = int(handLandmarks.landmark[10].x * w), int(handLandmarks.landmark[10].y * h)

        if y12 < y10:
            fingers['12'] = 1
        
        # ring finger
        x16, y16 = int(handLandmarks.landmark[16].x * w), int(handLandmarks.landmark[16].y * h)
        x14, y14 = int(handLandmarks.landmark[14].x * w), int(handLandmarks.landmark[14].y * h)

        if y16 < y14:
            fingers['16'] = 1
        
        # pinky finger
        x20, y20 = int(handLandmarks.landmark[20].x * w), int(handLandmarks.landmark[20].y * h)
        x18, y18 = int(handLandmarks.landmark[18].x * w), int(handLandmarks.landmark[18].y * h)
    
        if y20 < y18:
            fingers['20'] = 1

        mpDraw.draw_landmarks(frame, handLandmarks, mpHands.HAND_CONNECTIONS)

    # count raised fingers
    number_fingers = sum_dict(fingers)

    # draw finger counter visual
    cv2.rectangle(frame, (25, 150), (100, 400), (0, 128, 0), cv2.FILLED)
    cv2.putText(frame, str(number_fingers), (35, 300), cv2.FONT_HERSHEY_PLAIN,
                3, (0, 71, 71), 2)
    
    cv2.imshow('Output', frame)
    # if q is pressed, program exits
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1